In [1]:
from keras.datasets import mnist
import numpy as np
%matplotlib inline
# 我們會使用到一些內建的資料庫, MAC需要加入以下兩行, 才不會把對方的ssl憑證視為無效
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Using TensorFlow backend.


In [2]:
# 回傳值: ((訓練特徵, 訓練目標), (測試特徵, 測試目標))
(x_train, y_train),(x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
x_train.shape

(60000, 28, 28)

In [0]:
from keras.utils import np_utils
# reshape讓他從 32 * 32變成 784 * 1的一維陣列
# 讓我們標準化到-1~1區間
x_train_shaped = (x_train - 127.5)/127.5
x_test_shaped = (x_test - 127.5)/127.5

In [5]:
print('原本的y_train shape:', y_train.shape)
y_train

原本的y_train shape: (60000,)


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
# 這裡reshape比較特別的參數是-1
# nparray的reshape容許你填入一個-1
# -1是指會自動幫你算出-1該等於多少
# ex (2, 3) reshape (-1, 1) 相當於reshape成(6, 1)
y_train = y_train.reshape(-1, 1)
print('後來的y_train shape:', y_train.shape)
y_train

後來的y_train shape: (60000, 1)


array([[5],
       [0],
       [4],
       ...,
       [5],
       [6],
       [8]], dtype=uint8)

In [0]:
img_shape = (28, 28)
random_dim = 100

In [20]:
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers import Embedding, multiply, BatchNormalization, UpSampling2D, Conv2D
# 這裡跟我們GAN的創作家一模一樣
generator = Sequential()
# 先讓100隨機亂數可以變成 7 * 7 * 128
# 為何是 7 * 7呢?
# 因為 7 *7 -> (第一次轉置) 14 * 14 -> (第二次轉置) 28 * 28
# 128則是使用類似VGG的概念, 選擇128開始
generator.add(Dense(7 * 7 * 128, input_dim=random_dim, activation='relu'))
# 轉換成三維
generator.add(Reshape((7, 7, 128)))
# 上採樣, 長寬變兩倍
generator.add(UpSampling2D(size=(2, 2)))
# (4, 4)卷積窗的卷積, 之所以做(4, 4)是為了跟discriminator配合, 我們等discriminator再談
generator.add(Conv2D(128, kernel_size=(4, 4), activation='relu', padding='same'))
# 卷積層間我喜歡使用BN來normalize
generator.add(BatchNormalization())
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(4, 4), activation='relu', padding='same'))
generator.add(BatchNormalization())
# 最後讓filter數目回到1, 因為是灰階圖片, 最後輸出 28 * 28 * 1圖片
# 一樣使用tanh(-1 - 1)作為激活
generator.add(Conv2D(1, kernel_size=(4, 4), activation='tanh', padding='same'))
generator.add(Reshape((28, 28)))
generator.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_5 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 64)       

In [21]:
# 這裡必需使用Model來做比較複雜的模型
noise = Input(shape=(random_dim,))
# 我們的標籤輸入 輸入只有一個維度 (1, )的,是為了讓小括號被當成tuple
# 否則會被當成普通的()
label = Input(shape=(1,), dtype='int32')
# 使用Embedding得到100向量
# input_dim = 10 -> 0~9
# output_dim = 100 -> 100維度的向量
# 接著使用Flatten把(1, 100)轉化成為(100)
label_embedding = Flatten()(Embedding(input_dim = 10, output_dim = random_dim)(label))
# 把靈感和標籤乘起來
model_input = multiply([noise, label_embedding])
# 經過我們上面的Generator做出圖片
img = generator(model_input)
# 完整的generator
# inputs = [noise, label] -> [靈感, 標籤]
# outputs = img -> 創作圖片
cgenerator = Model([noise, label], img)
cgenerator.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 100)       1000        input_8[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 100)          0           embedding_3[0][0]                
____________________________________________________________________________________________

In [22]:

discriminator = Sequential()
# 步長2卷積
discriminator.add(Conv2D(32, kernel_size=4, 
                         strides=2, 
                         input_shape=(28, 28, 1), 
                         padding="same",
                         activation='relu'))
# 我一樣會在兩層卷積中加入BN
discriminator.add(BatchNormalization())
discriminator.add(Conv2D(64, kernel_size=4, 
                         strides=2, 
                         padding="same",
                         activation='relu'))
discriminator.add(BatchNormalization())
discriminator.add(Conv2D(128, kernel_size=4, 
                         strides=2, 
                         padding="same",
                         activation='relu'))
discriminator.add(BatchNormalization())
# 開始全連接層(MLP)
discriminator.add(Flatten())
discriminator.add(Dense(256, activation='relu'))
discriminator.add(Dropout(0.25))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 14, 14, 32)        544       
_________________________________________________________________
batch_normalization_8 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 7, 7, 64)          32832     
_________________________________________________________________
batch_normalization_9 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 4, 4, 128)         131200    
_________________________________________________________________
batch_normalization_10 (Batc (None, 4, 4, 128)         512       
_________________________________________________________________
flatten_6 (Flatten)          (None, 2048)             

In [23]:
img = Input(shape=img_shape)
# 得到標籤的 784 維度表示
label = Input(shape=(1,), dtype='int32')
label_embedding = Flatten()(Embedding(input_dim=10, output_dim=784)(label))
flat_img = Flatten()(img)
model_input = multiply([flat_img, label_embedding])
model_input = Reshape((28, 28, 1))(model_input)
validity = discriminator(model_input)
cdiscriminator = Model([img, label], validity)
cdiscriminator.compile(loss='binary_crossentropy', optimizer="adam")
cdiscriminator.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 784)       7840        input_10[0][0]                   
__________________________________________________________________________________________________
flatten_8 (Flatten)             (None, 784)          0           input_9[0][0]                    
____________________________________________________________________________________________

In [24]:
# 記得在組合網路的時候必須讓鑑賞家保持不動!
cdiscriminator.trainable = False
cgan_input = Input(shape=(random_dim,))
cgan_label = Input(shape=(1, ))
x = cgenerator([cgan_input, cgan_label])
cgan_output = cdiscriminator([x, cgan_label])
cgan = Model(inputs=[cgan_input, cgan_label], outputs=cgan_output)
cgan.compile(loss='binary_crossentropy', optimizer="adam")
cgan.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
model_4 (Model)                 (None, 28, 28)       1029673     input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
model_5 (Model)                 (None, 1)            698113      model_4[1][0]              

In [0]:
batch_size = 200
epoch_count = 100

for epoch in range(0, epoch_count):
    for batch_count in range(0, 300):
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train_shaped[idx]
        # 不一樣的點在這裡!我們有把訓練資料的標籤拿出來
        labels = y_train[idx]
        
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        # 步驟0:讓創作家製造出fake image
        noise = np.random.normal(0, 1, (batch_size, random_dim))
        # 跟普通GAN不一樣, 帶入了標籤部分
        gen_imgs = cgenerator.predict([noise, labels])
        # gen_imgs = Reshape((200, 28, 28))(gen_imgs)
        # 步驟1:讓鑑賞家鑑賞對的image
        d_loss_real = cdiscriminator.train_on_batch([imgs, labels], valid)
        # 步驟2:讓鑑賞家鑑賞錯的image
        d_loss_fake = cdiscriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = (d_loss_real + d_loss_fake) / 2

        noise = np.random.normal(0, 1, (batch_size, random_dim))
        # 步驟3:訓練創作家的創作能力
        g_loss = cgan.train_on_batch([noise, labels], valid)

    if (epoch + 1) % 10 == 0:
        dash = "-" * 15
        print(dash, "epoch", epoch + 1, dash)
        print("Discriminator loss:", d_loss)
        print("Generator loss:", g_loss)



/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


--------------- epoch 10 ---------------
Discriminator loss: 0.007914029061794281
Generator loss: 0.0881186


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
noise = np.random.normal(0, 1, (10, random_dim))
sampled_labels = np.arrang([0,9,6,3,0,3,2,5,5,0]).reshape(-1, 1)

gen_imgs = cgenerator.predict([noise, sampled_labels])

# Rescale images 0 - 1
gen_imgs = 0.5 * gen_imgs + 0.5
gen_imgs = gen_imgs.reshape(10, 28, 28)
plt.figure(figsize = (14, 14))
# range(0, 10)產生出十種不同label(0 - 9) 針對性的產生我們要的數字
for i in range(0, 10):
    plt.subplot(1, 10, i + 1)
    plt.axis("off")
    plt.imshow(gen_imgs[i], cmap='gray')